In [1]:
from mqt.bench import CompilerSettings, QiskitSettings, get_benchmark
from qiskit import *
import numpy as np

min_qubit = 3 # maximum number of qubit
max_qubit = 5 # minimum number of qubit
gaps = 1
min_qubit = min(max(2, min_qubit), max_qubit) # maximum number of qubit
max_qubit = max(2, max_qubit) # minimum number of qubit
#skip_qubis = max(1, skip_qubits)
gaps = 1
compiler_settings = CompilerSettings(qiskit=QiskitSettings(optimization_level=1))
circuits= []
for i in range(min_qubit, max_qubit, gaps):
    qc = get_benchmark(benchmark_name="grover-noancilla",
                       level="nativegates",
                       circuit_size=i,
                       compiler="qiskit",
                       compiler_settings=compiler_settings,
                       provider_name="ionq",)
    circuits.append((i,qc))
    print("---------------grover with qubit", i,"---------------")
print("=================================================================================================")
print(circuits)

---------------grover with qubit 3 ---------------
---------------grover with qubit 4 ---------------
[(3, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7d4fc0daa6c0>), (4, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7d505ef61910>)]


In [2]:
from qiskit_aer import AerSimulator
from qiskit import *
from qiskit.providers.fake_provider import *
import numpy as np 
import os
from qiskit.primitives import StatevectorSampler
sampler = AerSimulator()
jobs=[]
for qubit_number,circuit_object in circuits: 
    job = sampler.run([circuit_object], shots=1000)
    results = job.result()
    counts= results.get_counts()
    #print(f" > Counts: {results[0].data["meas"].get_counts()}")
    jobs.append((qubit_number , counts))
print(jobs)

[(3, {'111': 1000}), (4, {'1101': 10, '1011': 11, '1010': 5, '1000': 11, '1001': 13, '1100': 13, '1111': 927, '1110': 10})]


In [3]:
# ----------------------------
# Hellinger fidelity
# ----------------------------
def hellinger_fidelity_with_expected(p, q):
    """Compute Hellinger fidelity between measured distribution p (counts)
       and expected distribution q (counts)."""
    p_sum, q_sum = sum(p.values()), sum(q.values())
    if q_sum == 0:
        raise ValueError("Expected distribution invalid (all counts = 0)")
    p_norm = {k: v / p_sum for k, v in p.items()}
    q_norm = {k: v / q_sum for k, v in q.items()}
    overlap = 0.0
    for k in set(p_norm) | set(q_norm):
        overlap += np.sqrt(p_norm.get(k, 0) * q_norm.get(k, 0))
    return overlap**2

# ----------------------------
# Ideal Grover distribution
# ----------------------------
def grovers_dist(num_qubits, marked_item):   
    n_iterations = int(np.pi * np.sqrt(2 ** num_qubits) / 4)   
    dist = {}
    for i in range(2**num_qubits):
        key = bin(i)[2:].zfill(num_qubits)
        theta = np.arcsin(1/np.sqrt(2 ** num_qubits))       
        if i == int(marked_item):
            dist[key] = np.sin((2*n_iterations+1)*theta)**2
        else:
            dist[key] = (np.cos((2*n_iterations+1)*theta)/(np.sqrt(2 ** num_qubits - 1)))**2
    return dist


In [4]:
from qiskit_aer import AerSimulator
sampler = AerSimulator()
jobs=[]
for qubit_number,circuit_object in circuits: 
    job = sampler.run(transpile(circuit_object, sampler), shots=1000)
    results = job.result()
    counts= results.get_counts()
    jobs.append((qubit_number , counts))
print(jobs)


# ----------------------------
# Run fidelity evaluation
# ----------------------------
verbose = True
marked_item = 1   # example: target state "1"

for num_qubits, counts in jobs:
    if verbose: print(f"\nFor qubit {num_qubits}, measured: {counts}")
    correct_dist = grovers_dist(num_qubits, marked_item)
    if verbose: print(f"Expected dist: {correct_dist}")
    fidelity = hellinger_fidelity_with_expected(counts, correct_dist)
    print(f"Hellinger fidelity (q={num_qubits}): {fidelity:.4f}")


[(3, {'111': 1000}), (4, {'1011': 2, '1101': 4, '1110': 10, '1001': 9, '1100': 8, '1000': 6, '1010': 8, '1111': 953})]

For qubit 3, measured: {'111': 1000}
Expected dist: {'000': 0.007812499999999989, '001': 0.9453125000000001, '010': 0.007812499999999989, '011': 0.007812499999999989, '100': 0.007812499999999989, '101': 0.007812499999999989, '110': 0.007812499999999989, '111': 0.007812499999999989}
Hellinger fidelity (q=3): 0.0078

For qubit 4, measured: {'1011': 2, '1101': 4, '1110': 10, '1001': 9, '1100': 8, '1000': 6, '1010': 8, '1111': 953}
Expected dist: {'0000': 0.0025787353515625, '0001': 0.9613189697265625, '0010': 0.0025787353515625, '0011': 0.0025787353515625, '0100': 0.0025787353515625, '0101': 0.0025787353515625, '0110': 0.0025787353515625, '0111': 0.0025787353515625, '1000': 0.0025787353515625, '1001': 0.0025787353515625, '1010': 0.0025787353515625, '1011': 0.0025787353515625, '1100': 0.0025787353515625, '1101': 0.0025787353515625, '1110': 0.0025787353515625, '1111': 0.00